In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

In [2]:
dataset_path = "/home/ns1254/robomimic/datasets/lift/mh/image_v141_gb100.hdf5" 
f_org = h5py.File(dataset_path, "r+")
demos = list(f_org["data"].keys())

In [3]:
lengths=[]
demos_minmax={}
for demo_name in demos:
    demo=f_org['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

    action=f_org['data'][demo_name]['actions']
    action=np.array(action) 
    demos_minmax[demo_name] = (np.min(action, axis=0), np.max(action, axis=0))

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  100
Max length:  59
Min length:  41
Mean length:  49.32


In [4]:
demos_minmax['demo_1']

(array([-0.125     , -0.32      , -0.747     , -0.06348293, -0.04605475,
        -0.02007322, -1.        ]),
 array([0.744     , 0.184     , 1.        , 0.03319882, 0.24098656,
        0.13674498, 1.        ]))

In [5]:
demos_minmax['demo_51']

(array([-0.94384869, -0.37895294, -0.99081614, -0.08258314, -0.29114203,
        -0.55971844, -1.        ]),
 array([0.75818276, 0.35310605, 0.99914343, 0.09128128, 0.24266282,
        0.16261121, 1.        ]))

In [6]:
mins,maxs=[],[]
for demo_name in demos_minmax.keys():
    min,max= demos_minmax[demo_name]
    mins.append(min)
    maxs.append(max)

mins=np.min(mins, axis=0)
maxs=np.max(maxs, axis=0) + 1e-8


np.set_printoptions(precision=4, suppress=True)

print('Action min: ', mins)
print('Action max: ', maxs)

Action min:  [-0.9888 -0.3847 -1.     -0.0959 -0.2983 -0.5665 -1.    ]
Action max:  [0.989  0.385  1.     0.0957 0.2984 0.5668 1.    ]


In [ ]:
# Action min:  [-0.289  -0.322  -1.     -0.0635 -0.0776 -0.195  -1.    ]
# Action max:  [0.923  0.385  1.     0.0721 0.241  0.5668 1.    ]

In [12]:
f_org['mask'].keys()
for key in f_org['mask'].keys():
    print(key, f_org['mask'][key].shape)

bad (50,)
expert (50,)
g20 (20,)
g20b5c (25,)
g40 (40,)
g40b10c (50,)


In [8]:
expert=[demo.decode('utf-8') for demo in f_org['mask']['expert']]
demos=[demo.decode('utf-8') for demo in f_org['mask']['bad']]
len(expert), len(demos)

(50, 50)

In [10]:
gb=expert[:20] # + demos[:5]
len(gb)

20

In [11]:
hdf5_path=dataset_path

demos= np.array(gb, dtype='S16') 

filter_keys=sorted([elem for elem in demos])
filter_name="g20"
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [15]:
demos[:3]

['demo_100', 'demo_51', 'demo_52']

In [8]:
expert[:3], demos[:3]

(['demo_1', 'demo_10', 'demo_11'], ['demo_100', 'demo_51', 'demo_52'])

In [13]:
# gb

In [14]:
expert=[demo.decode('utf-8') for demo in f_org['mask']['g20']]
demos=[demo.decode('utf-8') for demo in f_org['mask']['g20b5c']]

In [15]:
expert

['demo_1',
 'demo_10',
 'demo_11',
 'demo_12',
 'demo_13',
 'demo_14',
 'demo_15',
 'demo_16',
 'demo_17',
 'demo_18',
 'demo_19',
 'demo_2',
 'demo_20',
 'demo_21',
 'demo_22',
 'demo_23',
 'demo_24',
 'demo_25',
 'demo_26',
 'demo_27']

In [16]:
demos

['demo_1',
 'demo_10',
 'demo_100',
 'demo_11',
 'demo_12',
 'demo_13',
 'demo_14',
 'demo_15',
 'demo_16',
 'demo_17',
 'demo_18',
 'demo_19',
 'demo_2',
 'demo_20',
 'demo_21',
 'demo_22',
 'demo_23',
 'demo_24',
 'demo_25',
 'demo_26',
 'demo_27',
 'demo_51',
 'demo_52',
 'demo_53',
 'demo_54']

In [23]:
mins=np.ones(7)*-1
maxs=np.ones(7)

print(mins)
print(maxs)

[-1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1.]


In [12]:
epsilon=0.8
for demo_name in demos:
    demo=f_org['data'][demo_name]  
    action_org=f_org['data'][demo_name]['actions']
    
    
    actions=[]
    for action in action_org:
        if np.random.rand() <= epsilon:
            action = -np.random.uniform(mins, maxs)
        actions.append(action)
    actions=np.array(actions)

    print(demo_name, actions.shape, action_org.shape)

    del f_org["data"][demo_name]['actions']
    f_org["data"][demo_name].create_dataset('actions', data=actions)

demo_100 (41, 7) (41, 7)
demo_51 (48, 7) (48, 7)
demo_52 (55, 7) (55, 7)
demo_53 (49, 7) (49, 7)
demo_54 (48, 7) (48, 7)
demo_55 (48, 7) (48, 7)
demo_56 (49, 7) (49, 7)
demo_57 (58, 7) (58, 7)
demo_58 (55, 7) (55, 7)
demo_59 (46, 7) (46, 7)
demo_60 (54, 7) (54, 7)
demo_61 (46, 7) (46, 7)
demo_62 (48, 7) (48, 7)
demo_63 (46, 7) (46, 7)
demo_64 (59, 7) (59, 7)
demo_65 (45, 7) (45, 7)
demo_66 (44, 7) (44, 7)
demo_67 (52, 7) (52, 7)
demo_68 (48, 7) (48, 7)
demo_69 (56, 7) (56, 7)
demo_70 (47, 7) (47, 7)
demo_71 (51, 7) (51, 7)
demo_72 (51, 7) (51, 7)
demo_73 (50, 7) (50, 7)
demo_74 (45, 7) (45, 7)
demo_75 (44, 7) (44, 7)
demo_76 (51, 7) (51, 7)
demo_77 (52, 7) (52, 7)
demo_78 (49, 7) (49, 7)
demo_79 (57, 7) (57, 7)
demo_80 (52, 7) (52, 7)
demo_81 (53, 7) (53, 7)
demo_82 (48, 7) (48, 7)
demo_83 (42, 7) (42, 7)
demo_84 (55, 7) (55, 7)
demo_85 (43, 7) (43, 7)
demo_86 (49, 7) (49, 7)
demo_87 (46, 7) (46, 7)
demo_88 (44, 7) (44, 7)
demo_89 (44, 7) (44, 7)
demo_90 (50, 7) (50, 7)
demo_91 (53, 7)

In [18]:
action.shape

(7,)

In [13]:
f_org.close()